In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
cudnn.benchmark = True
plt.ion()
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image
import torchvision.models as models
resnet50 = models.wide_resnet50_2(pretrained=True)
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = '/content/drive/MyDrive/Data_AI_T6/'

In [13]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.img_labels)
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.img_labels.iloc[idx, 1:].values.astype(int))    
        if self.transform:
            image = self.transform(image)
        return image, label

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [15]:
data_train =  CustomImageDataset(root_dir+'Train.csv',root_dir+'Train_img/x',transform = transform)
dataloader_train = torch.utils.data.DataLoader(data_train, batch_size = 30, shuffle=True,drop_last=True)
data_val =  CustomImageDataset(root_dir+'Val.csv',root_dir+'Val_img/x',transform = transform)
dataloader_val = torch.utils.data.DataLoader(data_val, batch_size = 30, shuffle=True,drop_last=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "gpu")

In [16]:
def calculate_accuracy(model,loader):
    correct = 0
    total = 0
    with torch.no_grad():
      for data in loader:
        images=data[0].to(device)
        labels=data[1].to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.max(labels, 1)[1]).sum().item()
    accuracy = 100 * correct / total
    return accuracy

def train_model(model, criterion, optimizer, scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    since = time.time()
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        print(f"Epoch: {epoch+1}/{num_epochs}\n-------")
        running_loss = 0.0
        for i,data in enumerate(dataloader_train,0):
            optimizer.zero_grad()
            inputs=data[0].to(device)
            labels=data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])    
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 50 == 49:
                print(f"Batch: {i+1}, Loss: {running_loss/50}")
                running_loss = 0.0
        accuracy = calculate_accuracy(model,dataloader_val)
        print(f"Validation Accuracy: {accuracy}%")
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), '/content/drive/MyDrive/Data_AI_T6/model2.pth')
    model.load_state_dict(best_model_wts)
    return model

In [19]:
class resnet(torch.nn.Module):
    def __init__(self, n_class):
        super(resnet, self).__init__()  
        resnet50 = models.resnet50(pretrained=True)
        base_layers = list(resnet50.children())  
        self.backbone = nn.Sequential(*base_layers[:-1])  
        self.cls = base_layers[-1]   
        num_ftrs = self.cls.in_features   
        self.cls = nn.Linear(num_ftrs,n_class) 
    def forward(self,x):
        x = self.backbone(x)
        x = x.view(x.size(0),-1 )  
        x = self.cls(x)
        
        return x

In [20]:
model_ft = resnet(7)
num_ftrs = model_ft.cls.in_features
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=10)


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch: 1/10
-------
Batch: 50, Loss: 1.1164340728521347
Batch: 100, Loss: 0.843417529463768
Batch: 150, Loss: 0.7596512621641159
Batch: 200, Loss: 0.7834904533624649
Batch: 250, Loss: 0.7016933476924896
Batch: 300, Loss: 0.693522475361824
Validation Accuracy: 78.33333333333333%
Epoch: 2/10
-------
Batch: 50, Loss: 0.6281402885913849
Batch: 100, Loss: 0.6429131162166596
Batch: 150, Loss: 0.6100650525093079
Batch: 200, Loss: 0.6133205020427703
Batch: 250, Loss: 0.5435804229974747
Batch: 300, Loss: 0.5562048888206482
Validation Accuracy: 86.11111111111111%
Epoch: 3/10
-------
Batch: 50, Loss: 0.43033059149980546
Batch: 100, Loss: 0.47082214698195457
Batch: 150, Loss: 0.49523948550224306
Batch: 200, Loss: 0.5434779191017151
Batch: 250, Loss: 0.4996171802282333
Batch: 300, Loss: 0.46840994358062743
Validation Accuracy: 85.0%
Epoch: 4/10
-------
Batch: 50, Loss: 0.3745983651280403
Batch: 100, Loss: 0.3722655454277992
Batch: 150, Loss: 0.39584810405969617
Batch: 200, Loss: 0.37514537051320074

In [22]:
class_labels = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
def predict_image(image_path, model):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((224, 224))
    img_tensor = ToTensor()(img).unsqueeze_(0)
    with torch.no_grad():
        model.eval()  
        output = model(img_tensor)
        predicted = torch.max(output.data, 1)
    return class_labels[predicted.indices]
model = resnet(7)
model.load_state_dict(torch.load('/content/drive/MyDrive/Data_AI_T6/model2.pth'))
model.eval()
test_dir = "/content/drive/MyDrive/Data_AI_T6/Test_img/x"
image_files = sorted([filename for filename in os.listdir(test_dir) if filename.endswith(".jpg")])
with open('/content/drive/MyDrive/Data_AI_T6/KetQua.csv', mode='w') as csv_file:
    fieldnames = ['image', 'MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for filename in image_files:
        image_path = os.path.join(test_dir, filename)
        predicted_label = predict_image(image_path,model)
        row = {'image': os.path.splitext(filename)[0]}  
        for i in range(len(class_labels)):
            if class_labels[i] == predicted_label:
                row[class_labels[i]] = 1.0
            else:
                row[class_labels[i]] = 0.0
        writer.writerow(row)
        print(f"The predicted label for {filename} is: {predicted_label}")

The predicted label for ISIC_0034524.jpg is: NV
The predicted label for ISIC_0034525.jpg is: NV
The predicted label for ISIC_0034526.jpg is: BCC
The predicted label for ISIC_0034527.jpg is: BCC
The predicted label for ISIC_0034528.jpg is: DF
The predicted label for ISIC_0034529.jpg is: BKL
The predicted label for ISIC_0034530.jpg is: NV
The predicted label for ISIC_0034531.jpg is: BCC
The predicted label for ISIC_0034532.jpg is: DF
The predicted label for ISIC_0034533.jpg is: DF
The predicted label for ISIC_0034534.jpg is: NV
The predicted label for ISIC_0034535.jpg is: BCC
The predicted label for ISIC_0034536.jpg is: BCC
The predicted label for ISIC_0034537.jpg is: NV
The predicted label for ISIC_0034538.jpg is: BCC
The predicted label for ISIC_0034539.jpg is: BCC
The predicted label for ISIC_0034540.jpg is: NV
The predicted label for ISIC_0034541.jpg is: DF
The predicted label for ISIC_0034542.jpg is: NV
The predicted label for ISIC_0034543.jpg is: DF
The predicted label for ISIC_003